## Imports

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import TumorDecon as td
from scipy.stats import wilcoxon
import seaborn as sns
from itertools import combinations
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree._tree import TREE_LEAF
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
import seaborn as sns
from scipy import stats
from sklearn.tree import _tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.tree._tree import TREE_LEAF
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
import seaborn as sns
from scipy import stats
from sklearn.tree import _tree
from itertools import combinations
from sklearn.neighbors import KernelDensity
from sklearn.cluster import KMeans
from collections import Counter
from itertools import combinations 
from statsmodels.stats.proportion import proportions_ztest
from sklearn.tree import DecisionTreeClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Data Formatting

In [2]:
gene_expression = pd.read_excel('C:/Users/NathanGrant/Programs/TripodsREU/data/uvm_tcga_pan_can_atlas_2018/data_RNA_Seq_v2_expression_median_excel.xlsx',index_col="Hugo_Symbol")
gene_expression = td.df_normalization(gene_expression, "minmax").T
presence_df = pd.read_csv('C:/Users/NathanGrant/Programs/TripodsREU/data/uvm_tcga_pan_can_atlas_2018/data_CNA.txt',delimiter='\t', index_col="Hugo_Symbol")
presence_df= presence_df[presence_df.columns.difference(['Entrez_Gene_Id','Cytoband'])]
immune_fractions_df = td.combine_celltypes(pd.read_excel("C:/Users/NathanGrant/Programs/TripodsREU/data/CibersortResults/UVMRun1/CIBERSORTOutput_Job2.xlsx",index_col="Input Sample"))
immune_fractions_df= immune_fractions_df[immune_fractions_df.columns.difference(['RMSE','P-value','Pearson Correlation'])]
immune_fractions_df_uncombined = pd.read_excel("C:/Users/NathanGrant/Programs/TripodsREU/data/CibersortResults/UVMRun1/CIBERSORTOutput_Job2.xlsx",index_col="Input Sample")
immune_fractions_df_uncombined= immune_fractions_df_uncombined[immune_fractions_df_uncombined.columns.difference(['RMSE','P-value','Pearson Correlation'])]
phenotypes = pd.read_csv("../data/TCGA-UVM.GDC_phenotype.tsv",delimiter="\t",index_col="submitter_id.samples")
phenotypes_2 = pd.read_csv('C:/Users/NathanGrant/Programs/TripodsREU/data/uvm_tcga_pan_can_atlas_2018/data_clinical_patient.txt',delimiter="\t",index_col="PATIENT_ID")


In [3]:
patient_dict = {}
for patient_id in list(gene_expression.index):
    patient_dict[patient_id] = {}
    patient_dict[patient_id]['BAP1_mutation'] = presence_df.loc['BAP1', patient_id]
    patient_dict[patient_id]['expression_values'] = gene_expression.loc[patient_id].values
    patient_dict[patient_id]['immune_fractions'] = immune_fractions_df.loc[patient_id].values
    patient_dict[patient_id]['DSS_time'] = phenotypes_2.loc[patient_id[:-3],'DSS_MONTHS']
    if phenotypes_2.loc[patient_id[:-3],'OS_STATUS'] == 'LIVING':
        patient_dict[patient_id]['os_status'] = 1
    else:
        patient_dict[patient_id]['os_status'] = 0
    if patient_id+"A" in list(phenotypes.index):
        patient_dict[patient_id]['tumor_size'] = phenotypes.loc[patient_id+"A",'tumor_basal_diameter']
    else:
        patient_dict[patient_id]['tumor_size'] = np.nan
    if patient_id+"A" in list(phenotypes.index):
        patient_dict[patient_id]['weight'] = phenotypes.loc[patient_id+"A",'weight']
    else:
        patient_dict[patient_id]['weight'] = np.nan
    if patient_id+"A" in list(phenotypes.index):
        patient_dict[patient_id]['age'] = phenotypes.loc[patient_id+"A",'age_at_initial_pathologic_diagnosis']
    else:
        patient_dict[patient_id]['age'] = np.nan
    if patient_id+"A" in list(phenotypes.index):
        if np.isnan(phenotypes.loc[patient_id+"A","tumor_thickness"]):
            patient_dict[patient_id]["tumor_thickness"] = np.nan
        else:
            patient_dict[patient_id]["tumor_thickness"] = phenotypes.loc[patient_id+"A","tumor_thickness"]
    else:
        patient_dict[patient_id]["tumor_thickness"] = np.nan
    if patient_id+"A" in list(phenotypes.index):
        s = phenotypes.loc[patient_id+"A","tumor_stage.diagnoses"]
        if s == "stage iv":
            patient_dict[patient_id]['stage'] = 2
        elif s == 'stage iiib' or s == 'stage iiic' or s == "stage iiia":
            patient_dict[patient_id]['stage'] = 1
        elif s == 'stage iib' or s == "stage iia":
            patient_dict[patient_id]['stage'] = 0
        else:
            patient_dict[patient_id]['stage'] = np.nan
    else:
        patient_dict[patient_id]['stage'] = np.nan
    if patient_id+"A" in list(phenotypes.index):
        patient_dict[patient_id]['bmi'] = phenotypes.loc[patient_id+"A",'bmi.exposures']
    else:
        patient_dict[patient_id]['bmi'] = np.nan
    